In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
plt.style.use("seaborn-whitegrid")
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import (OrdinalEncoder, StandardScaler, 
                                   MinMaxScaler, PolynomialFeatures,
                                   PowerTransformer)

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import Lasso, RidgeCV
from sklearn.ensemble import (RandomForestRegressor, AdaBoostRegressor,
                             GradientBoostingRegressor, ExtraTreesRegressor)
from sklearn.ensemble import VotingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.svm import SVC 
from xgboost import XGBRegressor
%matplotlib inline

from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV

import warnings
warnings.filterwarnings('ignore')


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
train=pd.read_csv("../input/jobathon-april-2022/train_E1GspfA.csv")
train.head()

In [4]:
train.isnull().sum()

In [7]:
test=pd.read_csv("../input/jobathon-april-2022/test_6QvDdzb.csv")
test.head()

In [9]:
sns.distplot(train.demand)

In [10]:
train['date']= pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
train

In [12]:
for df in [train, test]:
    df['weekday'] = df['date'].dt.weekday
    #df['hour'] = df.time.dt.hour
    #df['minute'] = df.time.dt.minute
    df['Day'] = df['date'].dt.day
    df['Month'] = df['date'].dt.month
    df['afternoon'] = (df['hour'] > 12).astype('int')
    df['weekend'] = df['date'].dt.dayofweek >=5
    

In [ ]:
def hours2timing(x):
    if x in [22,23,0,1,2,3]:
        timing = 'Night'
    elif x in range(4, 12):
        timing = 'Morning'
    elif x in range(12, 17):
        timing = 'Afternoon'
    elif x in range(17, 22):
        timing = 'Evening'
    else:
        timing = 'X'
    return timing


In [ ]:
train['timing'] = train['hour'].apply(hours2timing)
train.head(3)

In [ ]:
train.groupby("timing")['demand'].agg(['count','min','max','mean','median','sum']).T

In [13]:
feature=['hour', 'weekday', 'Day', 'Month','afternoon','weekend']

In [14]:
X=train[feature]
test1=test[feature]
test1

In [15]:
y=train.demand

In [16]:
def train1(model):
    
    clf = Pipeline(steps=[
                       ('model', model)
                     ])

    cv = KFold(n_splits=5, shuffle=True, random_state=1)
    ypred = 0
    total_loss = 0
    for train_indx, test_indx in cv.split(X):
        X_train, X_val = X.iloc[train_indx], X.iloc[test_indx]
        y_train, y_val = y.iloc[train_indx], y.iloc[test_indx]
        clf.fit(X_train, y_train)
        
        yhat = clf.predict(X_val)
        score =mean_squared_error(yhat, y_val,squared=False)
        print(f"Loss:{score}")
        ypred += clf.predict(test1) / 5
        total_loss += score / 5
        
    print(f"Avg. Loss: {total_loss}")     
    return ypred

In [18]:
model = LGBMRegressor(n_estimators=200)
#model=Lasso()
final_prediction = train1(model)


In [19]:
final_prediction

In [ ]:
train['timing']=train['timing'].astype('category').cat.codes
test['timing']=test['timing'].astype('category').cat.codes
train

In [21]:
sample_submission=pd.read_csv('../input/jobathon-april-2022/sample_4E0BhPN.csv')
sample_submission.head()

In [22]:
test['demand']=final_prediction
test.head()

In [ ]:
df2=test[['date','hour','demand']]
df2.to_csv('final_submission.csv',index=False)